In [ ]:
import torch
import torch.nn as nn

# Definimos un bloque residual
# que es una subestructura común en redes neuronales profundas.
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.2):
        super().__init__()
        # La red interna del bloque residual: 
        # dos capas lineales con normalización, activación y dropout.
        self.net = nn.Sequential(
            nn.Linear(dim, dim), # Capa lineal que mantiene la dimensionalidad
            nn.BatchNorm1d(dim), # Normalización por lotes para estabilizar el entrenamiento
            nn.GELU(),           # Activación no lineal GELU
            nn.Dropout(dropout), # Regularización para evitar sobreajuste
            nn.Linear(dim, dim), # Segunda capa lineal
            nn.BatchNorm1d(dim), # Segunda normalización por lotes
        )
        self.activation = nn.GELU() # Activación final después de la suma residual

    # La salida es la suma del input original (x) y
    # la salida de la red interna (self.net(x)),
    # seguida de una activación.
    def forward(self, x):
        return self.activation(x + self.net(x))


# Definimos el modelo principal:
# una red neuronal multicapa (MLP) para clasificación de emociones.
class EmotionMLP(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=128, depth=4, dropout=0.3):
        super().__init__()

        # Lista de capas iniciales: proyección de entrada a la dimensión oculta
        layers = [
            nn.Linear(input_dim, hidden_dim), # Proyecta la entrada a la dimensión oculta
            nn.BatchNorm1d(hidden_dim),       # Normalización por lotes
            nn.GELU(),                        # Activación no lineal
            nn.Dropout(dropout)               # Regularización
        ]

        # Agregamos varios bloques residuales según el parámetro `depth`
        for _ in range(depth):
            layers.append(ResidualBlock(hidden_dim, dropout=dropout))

        # Capa final: proyección de la dimensión oculta al número de clases
        layers.append(nn.Linear(hidden_dim, num_classes))
        
        # Creamos el modelo como una secuencia de capas
        self.model = nn.Sequential(*layers)

    # Propagación hacia adelante a través de todas las capas
    def forward(self, x):
        return self.model(x)


In [ ]:
input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))

model = EmotionMLP(input_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 30

for epoch in range(num_epochs):

    # FORWARD + LOSS
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # BACKPROP + UPDATE
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")
